# X4 Image to Image Translation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import h5py as h5
import torch as th

In [ ]:
path = '/mnt/data/datascience_miniprojects/04_ImageToImage/sim/'
path_inputs = path + 'inputs/'
path_targets = path + 'outputs/'

# %%
